In [16]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, mean_absolute_error
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

class PetActivityAnalyzer:
    def __init__(self):
        self.scaler = StandardScaler()
        self.feature_scaler = MinMaxScaler()
        self.lstm_model = None
        self.rf_model = None
        self.selected_features = None
        self.sequence_length = 24  # Default 24 time steps (e.g., 24 seconds)

    def preprocess_data(self, raw_data):
        """
        Preprocess raw sensor data from the Marshee device

        Parameters:
        raw_data (DataFrame): Raw accelerometer and gyroscope data
                             Expected columns: timestamp, acc_x, acc_y, acc_z, gyro_x, gyro_y, gyro_z

        Returns:
        DataFrame: Preprocessed data with engineered features
        """
        # Create a copy to avoid modifying the original
        data = raw_data.copy()

        # Ensure data is sorted by timestamp
        data = data.sort_values('timestamp')

        # Calculate magnitude of acceleration
        data['acc_magnitude'] = np.sqrt(data['acc_x']**2 + data['acc_y']**2 + data['acc_z']**2)

        # Calculate magnitude of angular velocity
        data['gyro_magnitude'] = np.sqrt(data['gyro_x']**2 + data['gyro_y']**2 + data['gyro_z']**2)

        # Apply moving average filter to reduce noise (window size of 5)
        data['acc_magnitude_smooth'] = data['acc_magnitude'].rolling(window=5, center=True).mean()
        data['gyro_magnitude_smooth'] = data['gyro_magnitude'].rolling(window=5, center=True).mean()

        # Fill NaN values created by rolling window
        data = data.bfill().ffill()

        # Calculate step frequency (time derivative of acceleration)
        data['acc_derivative'] = data['acc_magnitude_smooth'].diff() / data['timestamp'].diff().dt.total_seconds()

        # Calculate jerk (derivative of acceleration)
        data['jerk'] = data['acc_derivative'].diff() / data['timestamp'].diff().dt.total_seconds()

        # Calculate activity burst indicator (high acceleration periods)
        data['activity_burst'] = (data['acc_magnitude_smooth'] > data['acc_magnitude_smooth'].mean() +
                                 data['acc_magnitude_smooth'].std()).astype(int)

        # Feature: Rest periods (low activity periods)
        data['rest_period'] = (data['acc_magnitude_smooth'] < data['acc_magnitude_smooth'].mean() -
                              0.5 * data['acc_magnitude_smooth'].std()).astype(int)

        # Calculate rest duration (cumulative sum of consecutive rest periods)
        data['rest_group'] = (data['rest_period'].diff() != 0).cumsum()
        rest_durations = data[data['rest_period'] == 1].groupby('rest_group')['timestamp'].agg(['count']).reset_index()

        # Merge rest durations back
        data = data.merge(rest_durations, on='rest_group', how='left')
        data['rest_duration'] = data['count'].fillna(0)
        data = data.drop(['count', 'rest_group'], axis=1)

        # Generate time-based features
        data['hour_of_day'] = data['timestamp'].dt.hour
        data['day_of_week'] = data['timestamp'].dt.dayofweek

        # Calculate hour-based activity patterns
        hourly_activity = data.groupby('hour_of_day')['acc_magnitude_smooth'].mean().reset_index()
        data = data.merge(hourly_activity, on='hour_of_day', suffixes=('', '_hourly_avg'))

        # Calculate activity intensity (normalized acceleration magnitude)
        data['activity_intensity'] = data['acc_magnitude_smooth'] / data['acc_magnitude_smooth'].max()

        # Calculate activity regularity (std dev of acceleration in rolling windows)
        data['activity_regularity'] = data['acc_magnitude_smooth'].rolling(window=20).std()
        data['activity_regularity'] = data['activity_regularity'].bfill().ffill()

        return data

    def engineer_features(self, preprocessed_data, window_size=60):
        """
        Engineer features from preprocessed data using sliding windows

        Parameters:
        preprocessed_data (DataFrame): Output from preprocess_data
        window_size (int): Size of sliding window in seconds

        Returns:
        DataFrame: Feature-engineered data ready for model training
        """
        # Create a copy to avoid modifying the original
        data = preprocessed_data.copy()

        # Create time windows (assuming data is in seconds)
        data['time_window'] = (data['timestamp'] - data['timestamp'].min()).dt.total_seconds() // window_size

        # Aggregate features by time window
        features = data.groupby('time_window').agg({
            'acc_magnitude_smooth': ['mean', 'std', 'max', 'min'],
            'gyro_magnitude_smooth': ['mean', 'std', 'max'],
            'activity_burst': 'sum',
            'rest_period': 'sum',
            'rest_duration': 'max',
            'activity_intensity': ['mean', 'max'],
            'activity_regularity': 'mean',
            'acc_derivative': ['mean', 'std'],
            'jerk': ['mean', 'std', 'max']
        })

        # Flatten multi-index columns
        features.columns = ['_'.join(col).strip() for col in features.columns.values]

        # Calculate additional window-level features

        # Activity ratio (percentage of window spent active)
        features['activity_ratio'] = 1 - (features['rest_period_sum'] / window_size)

        # Activity variability (ratio of std to mean of acceleration)
        features['activity_variability'] = features['acc_magnitude_smooth_std'] / features['acc_magnitude_smooth_mean']

        # Replace infinity and NaN values
        features = features.replace([np.inf, -np.inf], np.nan)
        features = features.fillna(0)

        # Normalize features
        normalized_features = self.feature_scaler.fit_transform(features)
        normalized_features_df = pd.DataFrame(normalized_features, columns=features.columns)

        # Add time information back
        normalized_features_df['time_window'] = features.index

        return normalized_features_df

    def create_sequences(self, feature_data, sequence_length=None):
        """
        Create sequences for LSTM model

        Parameters:
        feature_data (DataFrame): Output from engineer_features
        sequence_length (int): Length of sequences to create

        Returns:
        tuple: (X, y) where X is sequence data and y is labels
        """
        if sequence_length is None:
            sequence_length = self.sequence_length
        else:
            self.sequence_length = sequence_length

        # Drop time_window from features for sequence creation
        features = feature_data.drop('time_window', axis=1).values

        X, y = [], []
        for i in range(len(features) - sequence_length):
            X.append(features[i:i + sequence_length])

            # For this example, we'll predict activity level (can be modified based on specific goals)
            # Using mean activity intensity of next time window as target
            target_idx = min(i + sequence_length, len(features) - 1)
            activity_idx = list(feature_data.columns).index('activity_intensity_mean')
            y.append(features[target_idx][activity_idx])

        return np.array(X), np.array(y)

    def feature_selection(self, feature_data, target_col='activity_intensity_mean', n_features=10):
        """
        Perform feature selection using Random Forest importance

        Parameters:
        feature_data (DataFrame): Feature engineered data
        target_col (str): Target column name
        n_features (int): Number of features to select

        Returns:
        list: Selected feature names
        """
        # Prepare data
        X = feature_data.drop(['time_window', target_col], axis=1)
        y = feature_data[target_col]

        # Initialize RF for feature importance - use RandomForestRegressor instead of Classifier
        rf = RandomForestRegressor(n_estimators=100, random_state=42)

        # Use RFE for feature selection
        rfe = RFE(estimator=rf, n_features_to_select=n_features)
        rfe.fit(X, y)

        # Get selected features
        selected_features = X.columns[rfe.support_].tolist()
        self.selected_features = selected_features

        return selected_features

    def build_lstm_model(self, input_shape, output_size=1):
        """
        Build LSTM model for time series prediction

        Parameters:
        input_shape (tuple): Shape of input data (sequence_length, n_features)
        output_size (int): Number of output nodes

        Returns:
        Model: Compiled Keras LSTM model
        """
        model = Sequential([
            LSTM(64, return_sequences=True, input_shape=input_shape),
            Dropout(0.2),
            BatchNormalization(),
            LSTM(32),
            Dropout(0.2),
            BatchNormalization(),
            Dense(16, activation='relu'),
            Dense(output_size, activation='linear')
        ])

        model.compile(
            optimizer='adam',
            loss='mean_squared_error',
            metrics=['mae']
        )

        self.lstm_model = model
        return model

    def train_lstm_model(self, X, y, epochs=100, batch_size=32, validation_split=0.2):
        """
        Train the LSTM model

        Parameters:
        X (numpy.array): Sequence data
        y (numpy.array): Target values
        epochs (int): Number of training epochs
        batch_size (int): Batch size for training
        validation_split (float): Portion of data to use for validation

        Returns:
        History: Training history
        """
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )

        checkpoint = ModelCheckpoint(
            'best_pet_activity_model.h5',
            monitor='val_loss',
            save_best_only=True
        )

        history = self.lstm_model.fit(
            X, y,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=validation_split,
            callbacks=[early_stopping, checkpoint],
            verbose=1
        )

        return history

    def build_rf_model(self):
        """
        Build Random Forest model for activity classification

        Returns:
        RandomForestClassifier: Initialized RF model
        """
        rf_model = RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            min_samples_split=5,
            min_samples_leaf=2,
            random_state=42
        )

        self.rf_model = rf_model
        return rf_model

    def train_rf_model(self, X, y):
        """
        Train Random Forest model

        Parameters:
        X (numpy.array): Feature data
        y (numpy.array): Target labels

        Returns:
        RandomForestClassifier: Trained RF model
        """
        self.rf_model.fit(X, y)
        return self.rf_model

    def prepare_for_deployment(self, quantize=True):
        """
        Prepare model for deployment to device

        Parameters:
        quantize (bool): Whether to quantize the model

        Returns:
        bytes: TFLite model ready for deployment
        """
        converter = tf.lite.TFLiteConverter.from_keras_model(self.lstm_model)

        # Add these experimental flags for TensorList handling
        converter.experimental_enable_resource_variables = True
        converter.target_spec.supported_ops = [
            tf.lite.OpsSet.TFLITE_BUILTINS,
            tf.lite.OpsSet.SELECT_TF_OPS
        ]
        converter._experimental_lower_tensor_list_ops = False

        if quantize:
            converter.optimizations = [tf.lite.Optimize.DEFAULT]
            converter.target_spec.supported_types = [tf.float16]

        try:
            tflite_model = converter.convert()
        except Exception as e:
            print(f"Conversion error: {e}")
            raise

        return tflite_model

    def evaluate_models(self, X_test, y_test, X_test_seq=None, y_test_seq=None):
        """
        Evaluate models on test data

        Parameters:
        X_test (numpy.array): Test features for RF model
        y_test (numpy.array): Test labels for RF model
        X_test_seq (numpy.array): Test sequence data for LSTM model
        y_test_seq (numpy.array): Test targets for LSTM model

        Returns:
        dict: Dictionary of evaluation metrics
        """
        results = {}

        # Evaluate RF model
        if self.rf_model is not None and X_test is not None and y_test is not None:
            y_pred = self.rf_model.predict(X_test)
            results['rf_accuracy'] = accuracy_score(y_test, y_pred)
            results['rf_f1'] = f1_score(y_test, y_pred, average='weighted')
            results['rf_confusion_matrix'] = confusion_matrix(y_test, y_pred)

        # Evaluate LSTM model
        if self.lstm_model is not None and X_test_seq is not None and y_test_seq is not None:
            lstm_preds = self.lstm_model.predict(X_test_seq)
            results['lstm_mae'] = mean_absolute_error(y_test_seq, lstm_preds)

            # Calculate RMSE
            results['lstm_rmse'] = np.sqrt(np.mean((y_test_seq - lstm_preds.flatten())**2))

            # Calculate R-squared
            ss_total = np.sum((y_test_seq - np.mean(y_test_seq))**2)
            ss_residual = np.sum((y_test_seq - lstm_preds.flatten())**2)
            results['lstm_r2'] = 1 - (ss_residual / ss_total)

        return results

    def classify_activity(self, feature_window):
        """
        Classify activity type from a window of features

        Parameters:
        feature_window (numpy.array): Window of features

        Returns:
        str: Activity classification
        float: Confidence score
        """
        if self.rf_model is None:
            raise ValueError("Random Forest model not trained. Call train_rf_model first.")

        # Ensure feature window is properly shaped and contains only selected features
        if self.selected_features is not None:
            # Extract only selected features if they exist in the input
            feature_cols = self.selected_features
        else:
            feature_cols = feature_window.columns

        # Get prediction
        prediction_prob = self.rf_model.predict_proba(feature_window[feature_cols].values.reshape(1, -1))
        prediction = self.rf_model.predict(feature_window[feature_cols].values.reshape(1, -1))

        # Map prediction to activity label
        activity_labels = ['rest', 'walking', 'running', 'playing']
        activity_type = activity_labels[prediction[0]] if prediction[0] < len(activity_labels) else 'unknown'

        # Get confidence
        confidence = np.max(prediction_prob)

        return activity_type, confidence

    def predict_future_activity(self, current_sequence):
        """
        Predict future activity level based on current sequence

        Parameters:
        current_sequence (numpy.array): Current sequence of features

        Returns:
        float: Predicted activity level
        """
        if self.lstm_model is None:
            raise ValueError("LSTM model not trained. Call train_lstm_model first.")

        # Ensure proper shape
        if len(current_sequence.shape) == 2:
            # Add batch dimension if needed
            current_sequence = np.expand_dims(current_sequence, axis=0)

        # Make prediction
        prediction = self.lstm_model.predict(current_sequence)

        return prediction[0][0]

    def detect_anomalies(self, feature_window, threshold=2.0):
        """
        Detect anomalies in activity patterns

        Parameters:
        feature_window (numpy.array): Window of features
        threshold (float): Threshold for anomaly detection (standard deviations)

        Returns:
        bool: True if anomaly detected, False otherwise
        dict: Anomaly details
        """
        # Get predictions
        activity_level = self.predict_future_activity(feature_window)

        # Compare with expected range
        expected_range = self.activity_baseline_stats.get('mean', 0.5) + threshold * self.activity_baseline_stats.get('std', 0.2)

        is_anomaly = abs(activity_level - self.activity_baseline_stats.get('mean', 0.5)) > expected_range

        details = {
            'predicted_activity': activity_level,
            'baseline_mean': self.activity_baseline_stats.get('mean', 0.5),
            'threshold': expected_range,
            'deviation': abs(activity_level - self.activity_baseline_stats.get('mean', 0.5))
        }

        return is_anomaly, details

    def set_activity_baseline(self, activity_data):
        """
        Set baseline statistics for anomaly detection

        Parameters:
        activity_data (numpy.array): Historical activity data

        Returns:
        dict: Baseline statistics
        """
        self.activity_baseline_stats = {
            'mean': np.mean(activity_data),
            'std': np.std(activity_data),
            'min': np.min(activity_data),
            'max': np.max(activity_data)
        }

        return self.activity_baseline_stats

    def visualize_activity_patterns(self, activity_data, timestamps):
        """
        Visualize activity patterns over time

        Parameters:
        activity_data (numpy.array): Activity intensity data
        timestamps (numpy.array): Corresponding timestamps

        Returns:
        matplotlib.figure.Figure: Figure object with visualization
        """
        fig, axes = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

        # Plot activity intensity over time
        axes[0].plot(timestamps, activity_data, 'b-', label='Activity Intensity')
        axes[0].set_title('Pet Activity Patterns')
        axes[0].set_ylabel('Activity Intensity')
        axes[0].legend()
        axes[0].grid(True)

        # Plot hourly patterns (boxplot)
        hourly_data = pd.DataFrame({'hour': pd.to_datetime(timestamps).hour, 'activity': activity_data})
        sns.boxplot(x='hour', y='activity', data=hourly_data, ax=axes[1])
        axes[1].set_title('Hourly Activity Distribution')
        axes[1].set_xlabel('Hour of Day')
        axes[1].set_ylabel('Activity Intensity')

        plt.tight_layout()
        return fig

# Main execution for demonstration
def generate_sample_data(num_samples=1000):
    """Generate sample accelerometer and gyroscope data for demonstration"""
    np.random.seed(42)

    # Create timestamps (one sample per second)
    start_time = pd.Timestamp('2023-01-01 08:00:00')
    timestamps = [start_time + pd.Timedelta(seconds=i) for i in range(num_samples)]

    # Create synthetic patterns (rest, walking, playing)
    patterns = np.random.choice(['rest', 'walking', 'playing'], size=num_samples, p=[0.3, 0.5, 0.2])

    # Generate accelerometer data based on patterns
    acc_x = np.zeros(num_samples)
    acc_y = np.zeros(num_samples)
    acc_z = np.zeros(num_samples)

    # Rest pattern (low activity)
    rest_idx = np.where(patterns == 'rest')[0]
    acc_x[rest_idx] = np.random.normal(0, 0.1, size=len(rest_idx))
    acc_y[rest_idx] = np.random.normal(0, 0.1, size=len(rest_idx))
    acc_z[rest_idx] = np.random.normal(1, 0.1, size=len(rest_idx))  # Gravity

    # Walking pattern (moderate activity)
    walk_idx = np.where(patterns == 'walking')[0]
    acc_x[walk_idx] = np.random.normal(0, 0.3, size=len(walk_idx))
    acc_y[walk_idx] = np.random.normal(0, 0.3, size=len(walk_idx))
    acc_z[walk_idx] = np.random.normal(1, 0.3, size=len(walk_idx))

    # Playing pattern (high activity)
    play_idx = np.where(patterns == 'playing')[0]
    acc_x[play_idx] = np.random.normal(0, 0.8, size=len(play_idx))
    acc_y[play_idx] = np.random.normal(0, 0.8, size=len(play_idx))
    acc_z[play_idx] = np.random.normal(1, 0.8, size=len(play_idx))

    # Generate gyroscope data
    gyro_x = np.zeros(num_samples)
    gyro_y = np.zeros(num_samples)
    gyro_z = np.zeros(num_samples)

    gyro_x[rest_idx] = np.random.normal(0, 0.05, size=len(rest_idx))
    gyro_y[rest_idx] = np.random.normal(0, 0.05, size=len(rest_idx))
    gyro_z[rest_idx] = np.random.normal(0, 0.05, size=len(rest_idx))

    gyro_x[walk_idx] = np.random.normal(0, 0.2, size=len(walk_idx))
    gyro_y[walk_idx] = np.random.normal(0, 0.2, size=len(walk_idx))
    gyro_z[walk_idx] = np.random.normal(0, 0.2, size=len(walk_idx))

    gyro_x[play_idx] = np.random.normal(0, 0.6, size=len(play_idx))
    gyro_y[play_idx] = np.random.normal(0, 0.6, size=len(play_idx))
    gyro_z[play_idx] = np.random.normal(0, 0.6, size=len(play_idx))

    # Create DataFrame
    data = pd.DataFrame({
        'timestamp': timestamps,
        'acc_x': acc_x,
        'acc_y': acc_y,
        'acc_z': acc_z,
        'gyro_x': gyro_x,
        'gyro_y': gyro_y,
        'gyro_z': gyro_z,
        'activity_type': patterns  # Ground truth for evaluation
    })

    return data

# Example usage
if __name__ == "__main__":
    # Generate sample data
    print("Generating sample data...")
    sample_data = generate_sample_data(10000)

    # Initialize analyzer
    analyzer = PetActivityAnalyzer()

    # Preprocess data
    print("Preprocessing data...")
    preprocessed_data = analyzer.preprocess_data(sample_data)

    # Add time_window to preprocessed_data using the same window_size as engineer_features
    window_size = 60  # Must match what's used in engineer_features()
    preprocessed_data['time_window'] = (
        (preprocessed_data['timestamp'] - preprocessed_data['timestamp'].min()).dt.total_seconds() // window_size
    )

    # Engineer features
    print("Engineering features...")
    feature_data = analyzer.engineer_features(preprocessed_data)

    # Create sequences for LSTM
    print("Creating sequences...")
    X_seq, y_seq = analyzer.create_sequences(feature_data)

    # Train/test split for sequences
    X_train_seq, X_test_seq, y_train_seq, y_test_seq = train_test_split(
        X_seq, y_seq, test_size=0.2, random_state=42
    )

    # Build and train LSTM model
    print("Building and training LSTM model...")
    input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])
    analyzer.build_lstm_model(input_shape)
    history = analyzer.train_lstm_model(X_train_seq, y_train_seq, epochs=50)

    # Select features and prepare for RF model
    print("Selecting features...")
    selected_features = analyzer.feature_selection(feature_data)

    # Extract activity types for classification
    activity_mapping = {'rest': 0, 'walking': 1, 'playing': 2}
    feature_data['activity_label'] = preprocessed_data.groupby('time_window')['activity_type'].agg(
        lambda x: activity_mapping.get(x.mode()[0], 0)  # Handle cases with no clear mode
    ).values

    # Prepare data for RF
    X_rf = feature_data[selected_features]
    y_rf = feature_data['activity_label']

    # Train/test split for RF
    X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(
        X_rf, y_rf, test_size=0.2, random_state=42
    )

    # Build and train RF model
    print("Building and training RF model...")
    analyzer.build_rf_model()
    analyzer.train_rf_model(X_train_rf, y_train_rf)

    # Evaluate models
    print("Evaluating models...")
    results = analyzer.evaluate_models(X_test_rf, y_test_rf, X_test_seq, y_test_seq)
    print("Evaluation results:", results)

    # Prepare for deployment
    print("Preparing model for deployment...")
    tflite_model = analyzer.prepare_for_deployment()

    # Save TFLite model
    with open('pet_activity_model.tflite', 'wb') as f:
        f.write(tflite_model)

    print("Model saved as pet_activity_model.tflite")

    # Set activity baseline for anomaly detection
    analyzer.set_activity_baseline(y_seq)

    print("Complete! The model is ready for integration with the Marshee app and device.")

Generating sample data...
Preprocessing data...
Engineering features...
Creating sequences...
Building and training LSTM model...
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/3 ━━━━━━━━━━━━━━━━━━━━ 9s 5s/step - loss: 3.9750 - mae: 1.5971

3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 356ms/step - loss: 3.6161 - mae: 1.5262 - val_loss: 0.5535 - val_mae: 0.7243
Epoch 2/50
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 2.9094 - mae: 1.3233

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 2.7107 - mae: 1.2714 - val_loss: 0.4808 - val_mae: 0.6723
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 2.2907 - mae: 1.3054

...

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.2414 - mae: 0.3953 - val_loss: 0.0490 - val_mae: 0.1832
Epoch 43/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.2055 - mae: 0.3688 - val_loss: 0.0521 - val_mae: 0.1906
Epoch 44/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.3059 - mae: 0.4370 - val_loss: 0.0561 - val_mae: 0.1996
Epoch 45/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.2317 - mae: 0.3892 - val_loss: 0.0543 - val_mae: 0.1969
Epoch 46/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.1855 - mae: 0.3428 - val_loss: 0.0513 - val_mae: 0.1910
Epoch 47/50
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.1615 - mae: 0.3212

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - loss: 0.1481 - mae: 0.3081 - val_loss: 0.0483 - val_mae: 0.1862
Epoch 48/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.1818 - mae: 0.3703 

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - loss: 0.1854 - mae: 0.3734 - val_loss: 0.0461 - val_mae: 0.1822
Epoch 49/50
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.1597 - mae: 0.3490

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 0.1806 - mae: 0.3616 - val_loss: 0.0436 - val_mae: 0.1701
Epoch 50/50
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - loss: 0.1907 - mae: 0.3565

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 0.1757 - mae: 0.3382 - val_loss: 0.0432 - val_mae: 0.1644
Selecting features...
Building and training RF model...
Evaluating models...
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 532ms/step
Evaluation results: {'rf_accuracy': 0.9705882352941176, 'rf_f1': 0.9561018437225637, 'rf_confusion_matrix': array([[ 0,  1],
       [ 0, 33]]), 'lstm_mae': 0.19660922690304056, 'lstm_rmse': np.float64(0.2621820594087152), 'lstm_r2': np.float64(-0.6283793307966079)}
Preparing model for deployment...
Saved artifact at '/tmp/tmpdbjlkgoh'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 24, 20), dtype=tf.float32, name='keras_tensor_54')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  134299027838672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134299027836368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134299027839440: TensorSpec(shape=(), dtype=tf.resource,